# Clean Current Labor Edge

In [1]:
import numpy as np
import pandas as pd
#import pyodbc
#from nltk import word_tokenize
#from spellchecker import SpellChecker
import re

import csv

### Temp Manual Update

In [3]:
import os
import sys

In [4]:
home_dir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, home_dir)
from util import db, nlp

In [5]:
conn = db.open_conn(is_prod=True)

In [4]:
new_labor = db.get_table_pd(f"SELECT * FROM dbo.Case_Detail WHERE Update_Date > '2021-04-16' and Detail_Notes is NULL", conn)

In [5]:
new_labor = new_labor.loc[new_labor.id > 100000]

In [6]:
new_labor.dropna(how='all', axis=1,inplace=True)

In [7]:
title_cols = ['item_description', 'product_family','oem_company', 'level']

new_labor = nlp.str_impute_lower(new_labor, title_cols)

# new_cols = db.get_columns(table_name = "Case_Detail",conn_tup = conn)

new_labor['detail_notes'] = new_labor[title_cols].agg(" ".join, axis=1)

token_full = nlp.tokenize_col(new_labor.detail_notes)
cleaned_matched_pos = nlp.normalize_tokens(token_full)
new_labor.detail_notes = [" ".join(row) for row in cleaned_matched_pos]

In [9]:
new_labor.to_csv("services_new.csv",index=False)

## Load Data from Database

In [3]:
from datetime import date
from datetime import timedelta

In [4]:
# Get datetime for SELECT query
yesterday = date.today() - timedelta(days = 1)
yes_str = yesterday.strftime('%Y-%m-%d')

In [16]:
#open connection to production db
conn = db.open_conn(is_prod=True)

In [53]:
new_labor = db.get_table_pd(f"SELECT * FROM dbo.Case_Detail WHERE Update_Date > '{yes_str}'", conn)

In [54]:
title_cols = ['item_description', 'product_family','oem_company', 'level']

new_labor = nlp.str_impute_lower(new_labor, title_cols)

new_cols = db.get_columns(table_name = "Case_Detail",conn_tup = conn)

new_labor.detail_notes = new_labor[title_cols].agg(" ".join, axis=1)

token_full = nlp.tokenize_col(new_labor.detail_notes)
cleaned_matched_pos = nlp.normalize_tokens(token_full)
new_labor.detail_notes = [" ".join(row) for row in cleaned_matched_pos]

In [63]:
conn.cnxn.close()

 Update Case_Detail Table

In [35]:
for i in range(len(new_rows)):
        #TODO: Full_Position_Title to Detail_Notes
    conn.cursor.execute(f"UPDATE {table} SET Full_Position_Title='{new_rows.full_position_title[i]}' WHERE ID = {new_rows.id[i]}")
    conn.cnxn.commit()
conn.cnxn.close()

## Generate Tech Dictionary

In [3]:
laboredge = str_impute_lower(laboredge)

laboredge['full_position'] = laboredge.position_title.str.lower()+" "+laboredge.product_category.fillna('').str.lower()

In [4]:
pos_tokens = tokenize_col(laboredge.full_position)

### Find misspelled words (occur > 1)

In [14]:
flat_toks = list(set([item for sublist in pos_tokens for item in sublist]))
stripped_toks = remove_punctuation(flat_toks)

In [15]:
misspelled = find_misspelled(stripped_toks)

misspelled = list(set(misspelled))

In [ ]:
tech_terms = word_occur_more(laboredge.full_position, misspelled)

In [17]:
write_list_csv('abbr_all.csv',tech_terms)

In [54]:
abbrev_ex = []
for ab in tech_terms:
    pattern = re.compile(r'(^|\W)'+ab+r'($|\W)')
    positions = laboredge.full_position[laboredge.full_position.str.contains(pattern)]
    abbrev_ex.append(positions)
    

In [55]:
pd.DataFrame({'term':tech_terms, 'example':abbrev_ex}).to_csv('tech_examples.csv', index=False)

#### Add Abbrevs to Spellchecker Dictionary

In [161]:
terms_corr = pd.read_excel('tech_examples.xlsx')

In [170]:
terms_corr['is_abbrev'] = terms_corr.term !=terms_corr.full_term

In [172]:
addition_words = list(terms_corr.term)
with open('tech_terms_pkl.txt', 'wb') as fh:
   pickle.dump(addition_words, fh)


In [175]:

ab_dict = {}
for i, row in terms_corr[terms_corr.is_abbrev].iterrows():
    ab_dict[row.term] = row.term+" "+row.full_term

r_num = {'i': '1',
         'ii': '2',
         'iii': '3',
         'iv': '4'}
ab_dict = dict(ab_dict, **r_num)

### Write Clean Position Titles and Tech Dictionary to pickles

In [176]:
# Create abbreviations dictionary pickle file
with open('pickles/abbrev_pkl.txt', 'wb') as fh:
   pickle.dump(ab_dict, fh)

In [183]:
sub_pos_tokens = normalize_tokens(pos_tokens)

In [188]:
# save cleaned positions tokenized
with open('pickles/positions_pkl.txt', 'wb') as fh:
   pickle.dump(sub_pos_tokens, fh)

#### Read from Pickle

In [5]:
clean_pos_tokens = pd.read_pickle(r'pickles/positions_pkl.txt')

In [6]:
join_positions = [" ".join(row) for row in clean_pos_tokens]

### Confirm Clean positions in same order

In [8]:
from fuzzywuzzy import fuzz

In [9]:
low_matches = []
for orig, clean in zip(laboredge.full_position, join_positions):
    if fuzz.token_sort_ratio(orig, clean) <70:
        low_matches.append([orig, clean])

In [10]:
len(low_matches)

1098

### Create new Laboredge with clean Positions

In [21]:
laboredge.full_position = join_positions
laboredge.drop(columns=['idxdontneed'], inplace=True)

In [22]:
laboredge.to_csv('labor_clean_pos.csv', index=False)